# Building a conversational chatbot using Rasa

## Import statements

In [16]:
import logging, json, warnings
warnings.filterwarnings('ignore')
logging.basicConfig(level='INFO')

import rasa.nlu
import rasa.core
import spacy

## NLU training data

In [30]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- yo
- yo what up
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- pce
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- later
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really
- not what I want

## intent: request_flight
- I want to go on a trip
- I want to book a flight
- I want to book a trip
- Need a ticket 
- Help me find a flight
- Get me out of here
- I want to travel
- I want to fly somewhere

## intent: inform
- from [Toronto] (place:origin)
- I'm coming from [New York] (place:origin)
- I'm going to [Florida] (place:destination)
- to [Tokyo] (place:destination)
- from [aug 10] (date:depart_date)
- leaving [may 4] (date:depart_date)
- to [aug 20] (date:return_date)
- come back [may 18] (date:return_date)
- budget is [$2000] (budget)
- no more than [1000] (budget)
- max [$400] (budget)
- [100 dollars] (budget)
- 100 - [200] (budget)
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


## NLU model configuration

In [31]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 
%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


## Train NLU model

In [32]:
from rasa.nlu.training_data import load_data
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Trainer
from rasa.nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa.nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.nlu.model:Starting to train component SpacyNLP
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SklearnIntentClassifier
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa

Fitting 2 folds for each of 6 candidates, totalling 12 fits


## Let's evaluate the NLU model with a sample text

In [33]:
def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("my budget is $1000"))

{
  "intent": {
    "name": "inform",
    "confidence": 0.31947327927665653
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "inform",
      "confidence": 0.31947327927665653
    },
    {
      "name": "goodbye",
      "confidence": 0.18536832891193372
    },
    {
      "name": "request_flight",
      "confidence": 0.1787035299373614
    },
    {
      "name": "deny",
      "confidence": 0.15855202924650416
    },
    {
      "name": "greet",
      "confidence": 0.08765807022611286
    },
    {
      "name": "affirm",
      "confidence": 0.07024476240143165
    }
  ],
  "text": "my budget is $1000"
}


## Evaluating NLU model on test data

(Definitely want to use unseen data, but will be using the data we have)

In [34]:
from rasa.nlu.test import run_evaluation

run_evaluation('nlu.md', model_directory)

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.nlu.test:Running model for predictions:
100%|██████████| 60/60 [00:00<00:00, 129.64it/s]
INFO:rasa.nlu.test:Intent evaluation results:
INFO:rasa.nlu.test:Intent Evaluation: Only considering those 60 examples that have a defined intent out of 60 examples
INFO:rasa.nlu.test:F1-Score:  0.8645524691358024
INFO:rasa.nlu.test:Precision: 0.8397603485838779
INFO:rasa.nlu.test:Accuracy:  0.9
INFO:rasa.nlu.test:Classification report: 
                precision    recall  f1-score   support

        inform       1.00      1.00      1.00        13
       goodbye       0.76      1.00      0.87        13
         greet       1.00      0.93      0.96        14
request_flight       1.00      1.00      1.00         8
        affirm       0.00      0.00      0.00         5
          deny       0.78      1.00      0.88         7

      accuracy                           0.90        60
     macro avg       0.76     

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5169613133002935},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5065495142929974},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5125129581708457},
   {'text': 'yo',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5069989987149721},
   {'text': 'yo what up',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.4723252405708644},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.49399432870478466},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.48707208843195393},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5244220259971187},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',


## Using Rasa core to teach the bot to respond

In [ ]:
stories_md = """
## flight path
* request_flight
    - utter_ask_to_where
* inform{'place':'destination'}
    - utter_ask_from_where 
* inform{'place':'origin'}
    - utter_ask_duration
* inform{'date':'depart_date','date':'return_date'}
    - utter_budget
* inform{'budget':'$1000'}
    - utter_flight_offer
    - utter_ok
* affirm
    - goodbye

## flight path 1
* request_flight
* inform{'place':'origin'}
* inform{'place':'destination'}
* inform{'date':'depart_date','date':'return_date'}
* inform{'budget':'$1000'}
    - utter_flight_offer
    - utter_ok
* affirm
    - goodbye



## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md
